# Get data

In [14]:
import numpy as np
import pandas as pd
import os
import yfinance as yf
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import fmin
from scipy.special import lambertw
from scipy.stats import kurtosis, zscore, normaltest, norm

from keras.utils.vis_utils import plot_model
from tensorflow import keras
from gan import GAN

from preprocess import Gaussianize, rolling_window

from tqdm.keras import TqdmCallback


In [2]:
raw_data = yf.download('^GSPC','2009-05-01','2018-12-31')

In [4]:
logret = np.diff(np.log(raw_data['Close']))
gauss = Gaussianize()
gauss.fit(logret)
u = gauss.transform(logret)

np.random.seed(0)

loc, scale = norm.fit(u)
print(f'loc: {loc:.6f}, scale: {scale:.6f}')
print(f'kurtosis: {kurtosis(u, fisher=False):.6f}')
print(normaltest(u))
xmin, xmax = u.min(), u.max()
x = np.linspace(xmin, xmax)
plt.plot(x, norm.pdf(x, loc, scale), label='theo')
plt.hist(u, alpha=0.5, bins=50, label='lamber', density=True)
plt.legend()
plt.show()



h:\My Drive\EPFL Account\ml4f\ml4f_project\src\preprocess.py:25: RuntimeWarning: invalid value encountered in true_divide
  return np.sign(z) * np.sqrt(np.real(lambertw(delta * z ** 2)) / delta)


3.962487043477273e-07 6


# Train model

In [7]:
data = rolling_window(u, 127, 5)

data = np.expand_dims(data, 2)
data.shape

batch_size = 128
data = tf.cast(data, tf.float32)
dataset = tf.data.Dataset.from_tensor_slices(data).batch(batch_size)

(462, 127, 1)

In [13]:
latent_size = 3
hidden_size = 80
output_size = 1
d_train_steps = 5
rfs = 127
gan = GAN(latent_size, hidden_size, output_size, d_train_steps)

In [9]:
g_optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0, beta_2=0.9)
d_optimizer = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0, beta_2=0.9)

gan.compile(
    d_optimizer=g_optimizer,
    g_optimizer=d_optimizer,
    loss_fn=keras.losses.BinaryCrossentropy()
)

history = gan.fit(dataset, epochs=100, verbose=0, callbacks=[TqdmCallback(verbose=1)])

In [10]:
history.history.keys()
plt.plot(history.history['g_loss'], label='g_loss')
plt.plot(history.history['d_loss'], label='d_loss')
plt.legend()
plt.show()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 127, 1)]     0           []                               
                                                                                                  
 temporal_block_7 (TemporalBloc  (None, 127, 80)     6642        ['input_1[0][0]']                
 k)                                                                                               
                                                                                                  
 temporal_block_8 (TemporalBloc  (None, 127, 80)     12962       ['temporal_block_7[0][0]']       
 k)                                                                                               
                                                                                              

In [ ]:
latent_noise = tf.random.normal(shape=(1, 3000, 3))
fake = gan.generator(latent_noise).numpy().reshape(3000)

In [ ]:
loc, scale = norm.fit(fake)
print(f'loc: {loc:.6f}, scale: {scale:.6f}')
print(f'kurtosis: {kurtosis(fake, fisher=False):.6f}')
plt.hist(fake, bins=50, alpha=0.5, label='fake')
plt.hist(u, bins=50, alpha=0.5, label='real')
plt.legend()
plt.show()